# 전처리

### 과정
#### Step1
- 박스오피스를 기준으로 2021년에 개봉한 영화들을 나열한다.
- 이를 기준으로 스크린당 관객수, 상영횟수당관객수를 추가하고, 이를 반영한다
- 상위 250개만 남기고 나머지는 지운다
- 위와 같은 과정으로 2018년까지 같은 과정을 반복한다.
- 모든 데이터가 나왔으면 합친다. 단, 중복된 경우가 있을 경우 하위년도를 기준으로 제거한다.

#### Step2
- 크롤링을 하여 각 영화마다 네티즌 평점, 전문가(평론가) 평점을 가져온다
- 각 데이터들의 리뷰데이터들을 가져와서 리뷰들을 감성분석하여 해당리뷰의 긍정도가 어느정도인지 지표로 나타내서 항목에 추가한다
- 크롤링된 데이터들을 저장하고, 영화 제목을 기준으로 중복된 항목들이 있거나, 평점이 등록되지 않은 경우는 제거
#### Step3
- 분류, 회귀 모델 등 여러 모델을 돌려보아 최적의 결과가 나오는 값으로 데이터를 산정한다.

### Step1 : 전처리

In [51]:
import pandas as pd
import numpy as np

In [52]:
movies2021=pd.read_excel('2021년 개봉 영화.xls')
box2021=pd.read_excel('2021년 박스오피스.xls')

In [53]:
box2021.drop(index=0,inplace=True)
box2021.rename(columns={'매출액' : '매출액점유율'},inplace=True)
box2021['장르']=box2021['장르 '].str.split(',').str[0]
box2021=box2021.drop(['장르 '],axis=1)
box2021.head(3)

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,감독,배우,장르
1,1,모가디슈,2021-07-28,3.419462e+10,0.089,3.419462e+10,3574448.0,3574448.0,1688.0,205723.0,한국,한국,"(주)덱스터스튜디오,(주)외유내강,(주)필름케이",롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,류승완,"김윤석,조인성,허준호,구교환,김소진,정만식,김재화,박경혜,박명신,한철우,주보비,안세...",액션
2,2,블랙 위도우,2021-07-07,2.999113e+10,0.078,2.999113e+10,2961678.0,2961678.0,2528.0,154982.0,미국,미국,NaN,월트디즈니컴퍼니코리아 유한책임회사,12세이상관람가,케이트 쇼트랜드,"스칼렛 요한슨,플로렌스 퓨,레이첼 와이즈,데이빗 하버",액션
3,3,분노의 질주: 더 얼티메이트,2021-05-19,2.205965e+10,0.058,2.205965e+10,2292413.0,2292413.0,2297.0,131856.0,미국,미국,NaN,유니버설픽쳐스인터내셔널 코리아(유),12세이상관람가,저스틴 린,"빈 디젤,존 시나,성 강,샤를리즈 테론,미셸 로드리게즈,조다나 브루스터,헬렌 미렌",액션


In [54]:
box2021=box2021.drop(['순위','누적매출액 ','배급사 ','감독 '],axis=1)

In [55]:
box2021.head(3)

,영화명,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,등급,배우,장르
1,모가디슈,2021-07-28,3.419462e+10,0.089,3574448.0,3574448.0,1688.0,205723.0,한국,한국,"(주)덱스터스튜디오,(주)외유내강,(주)필름케이",15세이상관람가,"김윤석,조인성,허준호,구교환,김소진,정만식,김재화,박경혜,박명신,한철우,주보비,안세...",액션
2,블랙 위도우,2021-07-07,2.999113e+10,0.078,2961678.0,2961678.0,2528.0,154982.0,미국,미국,NaN,12세이상관람가,"스칼렛 요한슨,플로렌스 퓨,레이첼 와이즈,데이빗 하버",액션
3,분노의 질주: 더 얼티메이트,2021-05-19,2.205965e+10,0.058,2292413.0,2292413.0,2297.0,131856.0,미국,미국,NaN,12세이상관람가,"빈 디젤,존 시나,성 강,샤를리즈 테론,미셸 로드리게즈,조다나 브루스터,헬렌 미렌",액션


In [56]:
s2021=pd.merge(box2021,movies2021)

In [57]:
s2021.head()

,영화명,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,...,등급,배우,장르,영화명(영문),제작연도,제작국가,유형,제작상태,감독,제작사
0,분노의 질주: 더 얼티메이트,2021-05-19,2.205965e+10,0.058,2292413.0,2292413.0,2297.0,131856.0,미국,미국,...,12세이상관람가,"빈 디젤,존 시나,성 강,샤를리즈 테론,미셸 로드리게즈,조다나 브루스터,헬렌 미렌",액션,Fast & Furious 9 THE FAST SAGA,2021,미국,장편,개봉,저스틴 린,NaN
1,싱크홀,2021-08-11,2.135107e+10,0.056,2187753.0,2187753.0,1603.0,121634.0,한국,한국,...,12세이상관람가,"차승원,김성균,이광수,권소현,김혜준,김재화,박옥출,남다름,정영숙,이학주,한태린,김건...",코미디,Sinkhole,2021,한국,장편,개봉,김지훈,(주)더타워픽쳐스
2,극장판 귀멸의 칼날: 무한열차편,2021-01-27,2.065604e+10,0.054,2151861.0,2151861.0,900.0,151898.0,일본,일본,...,15세이상관람가,"하나에 나츠키,키토 아카리,시모노 히로,마츠오카 요시츠구,히노 사토시,히라카와 다이스케",애니메이션,Demon Slayer: Kimetsu no Yaiba the Movie - Mug...,2020,일본,장편,개봉,소토자키 하루오,NaN
3,소울,2021-01-20,1.903454e+10,0.05,2048175.0,2048175.0,2018.0,160727.0,미국,미국,...,전체관람가,"제이미 폭스,티나 페이",애니메이션,SOUL,2020,미국,장편,개봉,피트 닥터,"월트 디즈니 픽쳐스,픽사 애니메이션 스튜디오"
4,미나리,2021-03-03,1.021116e+10,0.027,1134044.0,1134044.0,1162.0,112894.0,미국,미국,...,12세이상관람가,"스티븐 연,한예리,윤여정,앨런 김,노엘 조,윌 패튼",드라마,Minari,2020,미국,장편,개봉,정이삭,NaN


In [58]:
box2021.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3620 entries, 1 to 3620
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   영화명     3619 non-null   object        
 1   개봉일     2271 non-null   datetime64[ns]
 2   매출액     3620 non-null   float64       
 3   매출액점유율  3619 non-null   object        
 4   관객수     3620 non-null   float64       
 5   누적관객수   3620 non-null   float64       
 6   스크린수    3620 non-null   float64       
 7   상영횟수    3620 non-null   float64       
 8   대표국적    3588 non-null   object        
 9   국적      3588 non-null   object        
 10  제작사     1137 non-null   object        
 11  등급      2497 non-null   object        
 12  배우      2178 non-null   object        
 13  장르      3575 non-null   object        
dtypes: datetime64[ns](1), float64(5), object(8)
memory usage: 424.2+ KB


In [59]:
movies2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1396 entries, 0 to 1395
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   영화명      1396 non-null   object
 1   영화명(영문)  592 non-null    object
 2   제작연도     1396 non-null   int64 
 3   제작국가     1396 non-null   object
 4   유형       1396 non-null   object
 5   장르       1396 non-null   object
 6   제작상태     1396 non-null   object
 7   감독       1208 non-null   object
 8   제작사      617 non-null    object
dtypes: int64(1), object(8)
memory usage: 98.3+ KB


In [60]:
s2021.columns

Index(['영화명', '개봉일', '매출액 ', '매출액점유율', '관객수 ', '누적관객수 ', '스크린수 ', '상영횟수 ',
       '대표국적 ', '국적 ', '제작사 ', '등급 ', '배우 ', '장르', '영화명(영문)', '제작연도', '제작국가',
       '유형', '제작상태', '감독', '제작사'],
      dtype='object')

In [61]:
s2021=s2021.drop(['국적 ','배우 ','영화명(영문)','제작상태','제작사'],axis=1)
s2021.head()

,영화명,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수,대표국적,제작사,등급,장르,제작연도,제작국가,유형,감독
0,분노의 질주: 더 얼티메이트,2021-05-19,2.205965e+10,0.058,2292413.0,2292413.0,2297.0,131856.0,미국,NaN,12세이상관람가,액션,2021,미국,장편,저스틴 린
1,싱크홀,2021-08-11,2.135107e+10,0.056,2187753.0,2187753.0,1603.0,121634.0,한국,(주)더타워픽쳐스,12세이상관람가,코미디,2021,한국,장편,김지훈
2,극장판 귀멸의 칼날: 무한열차편,2021-01-27,2.065604e+10,0.054,2151861.0,2151861.0,900.0,151898.0,일본,NaN,15세이상관람가,애니메이션,2020,일본,장편,소토자키 하루오
3,소울,2021-01-20,1.903454e+10,0.05,2048175.0,2048175.0,2018.0,160727.0,미국,"월트 디즈니 픽쳐스,픽사 애니메이션 스튜디오",전체관람가,애니메이션,2020,미국,장편,피트 닥터
4,미나리,2021-03-03,1.021116e+10,0.027,1134044.0,1134044.0,1162.0,112894.0,미국,NaN,12세이상관람가,드라마,2020,미국,장편,정이삭


In [62]:
s2021.columns

Index(['영화명', '개봉일', '매출액 ', '매출액점유율', '관객수 ', '누적관객수 ', '스크린수 ', '상영횟수 ',
       '대표국적 ', '제작사 ', '등급 ', '장르', '제작연도', '제작국가', '유형', '감독'],
      dtype='object')

In [63]:
s2021=s2021.drop(['제작사 ','제작국가','누적관객수 '],axis=1)
s2021['편당관객수']=s2021['관객수 ']/s2021['상영횟수 ']
s2021['스크린당관객수']=s2021['관객수 ']/s2021['스크린수 ']
s2021.head()

,영화명,개봉일,매출액,매출액점유율,관객수,스크린수,상영횟수,대표국적,등급,장르,제작연도,유형,감독,편당관객수,스크린당관객수
0,분노의 질주: 더 얼티메이트,2021-05-19,2.205965e+10,0.058,2292413.0,2297.0,131856.0,미국,12세이상관람가,액션,2021,장편,저스틴 린,17.385731,998.003047
1,싱크홀,2021-08-11,2.135107e+10,0.056,2187753.0,1603.0,121634.0,한국,12세이상관람가,코미디,2021,장편,김지훈,17.986361,1364.786650
2,극장판 귀멸의 칼날: 무한열차편,2021-01-27,2.065604e+10,0.054,2151861.0,900.0,151898.0,일본,15세이상관람가,애니메이션,2020,장편,소토자키 하루오,14.166487,2390.956667
3,소울,2021-01-20,1.903454e+10,0.05,2048175.0,2018.0,160727.0,미국,전체관람가,애니메이션,2020,장편,피트 닥터,12.743192,1014.952924
4,미나리,2021-03-03,1.021116e+10,0.027,1134044.0,1162.0,112894.0,미국,12세이상관람가,드라마,2020,장편,정이삭,10.045211,975.941480


In [67]:
s2021=s2021.iloc[:250,:]
s2021.shape

(250, 15)

In [68]:
# 2018~2020년의 데이터들도 갖고와본다.
movies2020=pd.read_excel('2020년 개봉 영화.xls')
box2020=pd.read_excel('2020년 박스오피스.xls')

In [69]:
box2020.drop(index=0,inplace=True)
box2020.rename(columns={'매출액' : '매출액점유율'},inplace=True)
box2020['장르']=box2020['장르 '].str.split(',').str[0]
box2020=box2020.drop(['장르 '],axis=1)
box2020.head(3)

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,감독,배우,장르
1,1,남산의 부장들,2020-01-22,4.122522e+10,0.081,4.122522e+10,4750345.0,4750345.0,1659.0,140051.0,한국,한국,(주)하이브미디어코프,(주)쇼박스,15세이상관람가,우민호,"이병헌,이성민,곽도원,이희준,김소진,김승훈,박성근,서현우,지현준,박지일,이태형,주석...",드라마
2,2,다만 악에서 구하소서,2020-08-05,3.860226e+10,0.076,3.860226e+10,4357803.0,4357803.0,1998.0,193842.0,한국,한국,(주)하이브미디어코프,(주)씨제이이엔엠,15세이상관람가,홍원찬,"황정민,이정재,박정민,박소이,최희서,송영창,이서환,박명훈,오대환,심영은,김성곤,최정...",범죄
3,3,반도,2020-07-15,3.307253e+10,0.065,3.307253e+10,3812250.0,3812250.0,2575.0,199048.0,한국,한국,(주)영화사레드피터,(주)넥스트엔터테인먼트월드(NEW),15세이상관람가,연상호,"강동원,이정현,권해효,김민재,구교환,김도윤,이레,이예원,김규백,황영희,장소연,문우진...",액션


In [70]:
box2020=box2020.drop(['순위','누적매출액 ','배급사 ','감독 '],axis=1)
s2020=pd.merge(box2020,movies2020)

In [71]:
s2020=s2020.drop(['국적 ','배우 ','영화명(영문)','제작상태','제작사'],axis=1)
s2020.head()

,영화명,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수,대표국적,제작사,등급,장르,제작연도,제작국가,유형,감독
0,남산의 부장들,2020-01-22,4.122522e+10,0.081,4750345.0,4750345.0,1659.0,140051.0,한국,(주)하이브미디어코프,15세이상관람가,드라마,2019.0,한국,장편,우민호
1,#살아있다,2020-06-24,1.596822e+10,0.031,1903992.0,1903992.0,1882.0,137073.0,한국,"영화사 집,(주)퍼스펙티브픽쳐스",15세이상관람가,드라마,2019.0,한국,장편,조일형
2,담보,2020-09-29,1.474887e+10,0.029,1719521.0,1719521.0,1342.0,110260.0,한국,"(주)제이케이필름,(주)레드로버,(주)씨제이이엔엠,(주)영화사연",12세이상관람가,드라마,2020.0,한국,장편,강대규
3,삼진그룹 영어토익반,2020-10-21,1.398141e+10,0.027,1571286.0,1571286.0,1615.0,139275.0,한국,더램프(주),12세이상관람가,드라마,2020.0,한국,장편,이종필
4,정직한 후보,2020-02-12,1.285430e+10,0.025,1538038.0,1538038.0,1187.0,89425.0,한국,"(주)수필름,(주)홍필름",12세이상관람가,코미디,2019.0,한국,장편,장유정


In [72]:
s2020=s2020.drop(['제작사 ','제작국가','누적관객수 '],axis=1)
s2020['편당관객수']=s2020['관객수 ']/s2020['상영횟수 ']
s2020['스크린당관객수']=s2020['관객수 ']/s2020['스크린수 ']
s2020.head()

,영화명,개봉일,매출액,매출액점유율,관객수,스크린수,상영횟수,대표국적,등급,장르,제작연도,유형,감독,편당관객수,스크린당관객수
0,남산의 부장들,2020-01-22,4.122522e+10,0.081,4750345.0,1659.0,140051.0,한국,15세이상관람가,드라마,2019.0,장편,우민호,33.918680,2863.378541
1,#살아있다,2020-06-24,1.596822e+10,0.031,1903992.0,1882.0,137073.0,한국,15세이상관람가,드라마,2019.0,장편,조일형,13.890350,1011.685441
2,담보,2020-09-29,1.474887e+10,0.029,1719521.0,1342.0,110260.0,한국,12세이상관람가,드라마,2020.0,장편,강대규,15.595148,1281.312221
3,삼진그룹 영어토익반,2020-10-21,1.398141e+10,0.027,1571286.0,1615.0,139275.0,한국,12세이상관람가,드라마,2020.0,장편,이종필,11.281896,972.932508
4,정직한 후보,2020-02-12,1.285430e+10,0.025,1538038.0,1187.0,89425.0,한국,12세이상관람가,코미디,2019.0,장편,장유정,17.199195,1295.735468


In [73]:
s2020=s2020.iloc[:250,:]
s2020.shape

(250, 15)

In [74]:
movies2019=pd.read_excel('2019년 개봉 영화.xls')
box2019=pd.read_excel('2019년 박스오피스.xls')

box2019.drop(index=0,inplace=True)
box2019.rename(columns={'매출액' : '매출액점유율'},inplace=True)
box2019['장르']=box2019['장르 '].str.split(',').str[0]
box2019=box2019.drop(['장르 '],axis=1)

In [75]:
box2019=box2019.drop(['순위','누적매출액 ','배급사 ','감독 '],axis=1)
s2019=pd.merge(box2019,movies2019)
s2019=s2019.drop(['국적 ','배우 ','영화명(영문)','제작상태','제작사'],axis=1)
s2019.head()

,영화명,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수,대표국적,제작사,등급,장르,제작연도,제작국가,유형,감독
0,극한직업,2019-01-23,1.396518e+11,0.073,16265618.0,16265618.0,2003.0,292584.0,한국,"(주)어바웃필름,영화사 해그림 주식회사,(주)씨제이이엔엠",15세이상관람가,코미디,2018,한국,장편,이병헌
1,겨울왕국 2,2019-11-21,1.115962e+11,0.058,13369064.0,13369064.0,2648.0,282557.0,미국,NaN,전체관람가,애니메이션,2019,미국,장편,"크리스 벅,제니퍼 리"
2,기생충,2019-05-30,8.588396e+10,0.045,10085275.0,10085275.0,1948.0,192855.0,한국,(주)바른손이앤에이,15세이상관람가,드라마,2019,한국,장편,봉준호
3,82년생 김지영,2019-10-23,3.029854e+10,0.016,3678156.0,3678156.0,1486.0,145441.0,한국,(주)봄바람영화사,12세이상관람가,드라마,2019,한국,장편,김도영
4,분노의 질주: 홉스&쇼,2019-08-14,3.224177e+10,0.017,3657536.0,3657536.0,1311.0,94896.0,미국,NaN,12세이상관람가,액션,2019,미국,장편,데이빗 레이치


In [76]:
s2019=s2019.drop(['제작사 ','제작국가','누적관객수 '],axis=1)
s2019['편당관객수']=s2019['관객수 ']/s2019['상영횟수 ']
s2019['스크린당관객수']=s2019['관객수 ']/s2019['스크린수 ']
s2019=s2019.iloc[:250,:]
s2019.shape

(250, 15)

In [77]:
movies2018=pd.read_excel('2018년 개봉 영화.xls')
box2018=pd.read_excel('2018년 박스오피스.xls')

box2018.drop(index=0,inplace=True)
box2018.rename(columns={'매출액' : '매출액점유율'},inplace=True)
box2018['장르']=box2018['장르 '].str.split(',').str[0]
box2018=box2018.drop(['장르 '],axis=1)

In [78]:
box2018=box2018.drop(['순위','누적매출액 ','배급사 ','감독 '],axis=1)
s2018=pd.merge(box2018,movies2018)
s2018=s2018.drop(['국적 ','배우 ','영화명(영문)','제작상태','제작사'],axis=1)
s2018=s2018.drop(['제작사 ','제작국가','누적관객수 '],axis=1)
s2018['편당관객수']=s2018['관객수 ']/s2018['상영횟수 ']
s2018['스크린당관객수']=s2018['관객수 ']/s2018['스크린수 ']
s2018=s2018.iloc[:250,:]
s2018.shape

(250, 15)

In [79]:
movies=pd.concat([s2018,s2019,s2020,s2021])
movies.shape

(1000, 15)

In [80]:
movies

,영화명,개봉일,매출액,매출액점유율,관객수,스크린수,상영횟수,대표국적,등급,장르,제작연도,유형,감독,편당관객수,스크린당관객수
0,보헤미안 랩소디,2018-10-31,8.001044e+10,0.044,9224582.0,1179.0,204537.0,미국,12세이상관람가,드라마,2018.0,장편,브라이언 싱어,45.099821,7824.072943
1,공작,2018-08-08,4.278885e+10,0.024,4974520.0,1317.0,99001.0,한국,12세이상관람가,드라마,2017.0,장편,윤종빈,50.247169,3777.160213
2,국가부도의 날,2018-11-28,3.084586e+10,0.017,3747952.0,1180.0,100804.0,한국,12세이상관람가,드라마,2018.0,장편,최국희,37.180588,3176.230508
3,코코,2018-01-11,2.768548e+10,0.015,3510017.0,1177.0,85925.0,미국,전체관람가,애니메이션,2017.0,장편,리 언크리치,40.849776,2982.172472
4,아쿠아맨,2018-12-19,3.045429e+10,0.017,3491857.0,1285.0,54505.0,미국,12세이상관람가,액션,2018.0,장편,제임스 완,64.064893,2717.398444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,아웃브레이크,2021-07-29,5.000000e+05,0,100.0,5.0,5.0,캐나다,15세이상관람가,스릴러,2020.0,장편,NaN,20.000000,20.000000
246,엑소시즘: 악령의 눈동자,2021-09-08,4.560000e+05,0,100.0,5.0,14.0,미국,청소년관람불가,공포(호러),2019.0,장편,NaN,7.142857,20.000000
247,시니어퀸 : 58년 개띠 여고동창생,2021-05-11,6.930000e+05,0,99.0,1.0,1.0,한국,12세이상관람가,드라마,2020.0,장편,김문옥,99.000000,99.000000
248,보통날,2021-07-01,3.340000e+05,0,98.0,1.0,14.0,한국,15세이상관람가,드라마,2021.0,장편,"김예나,박규은,이미소",7.000000,98.000000


In [81]:
movies.columns

Index(['영화명', '개봉일', '매출액 ', '매출액점유율', '관객수 ', '스크린수 ', '상영횟수 ', '대표국적 ',
       '등급 ', '장르', '제작연도', '유형', '감독', '편당관객수', '스크린당관객수'],
      dtype='object')

In [82]:
movies['매출액 ']=np.log(movies['매출액 '])
movies.head(3)

C:\anaconda\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,영화명,개봉일,매출액,매출액점유율,관객수,스크린수,상영횟수,대표국적,등급,장르,제작연도,유형,감독,편당관객수,스크린당관객수
0,보헤미안 랩소디,2018-10-31,25.105423,0.044,9224582.0,1179.0,204537.0,미국,12세이상관람가,드라마,2018.0,장편,브라이언 싱어,45.099821,7824.072943
1,공작,2018-08-08,24.479543,0.024,4974520.0,1317.0,99001.0,한국,12세이상관람가,드라마,2017.0,장편,윤종빈,50.247169,3777.160213
2,국가부도의 날,2018-11-28,24.152268,0.017,3747952.0,1180.0,100804.0,한국,12세이상관람가,드라마,2018.0,장편,최국희,37.180588,3176.230508


### step2 : 크롤링

In [84]:
movies['영화명'].iloc[0]

'보헤미안 랩소디'

In [88]:
from selenium import webdriver
import time

In [89]:
options=webdriver.ChromeOptions()
options.add_argument('user_agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36')
options.add_experimental_option("excludeSwitches", ["enable-logging"])

basepath='C:/Users/이신행/OneDrive/바탕 화면/chromedriver_win32/'
browser=webdriver.Chrome(executable_path=basepath+'chromedriver.exe',options=options)

In [94]:
net_score=[]
spec_score=[]
for i in range(len(movies)):
    moviename=movies['영화명'].iloc[i]
    url='https://movie.naver.com/'
    browser.get(url)
    element=browser.find_element_by_id('ipt_tx_srch')
    element.send_keys(moviename)
    element2=browser.find_element_by_class_name('btn_srch')
    element2.click()
    browser.implicitly_wait(3)
#     element=browser.find_element_by_partial_link_text(moviename)
    element=browser.find_element_by_css_selector('#old_content > ul.search_list_1 > li > p > a > img')
    element.click()
    score1=browser.find_element_by_css_selector('div.netizen_score > div > div > em').text
    score2=browser.find_element_by_css_selector('div.special_score > div > div > em').text
    net_score.append(score1) 
    spec_score.append(score2)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#old_content > ul.search_list_1 > li > p > a > img"}
  (Session info: chrome=95.0.4638.69)


In [96]:
net_score

['9.42',
 '6.83',
 '8.12',
 '9.32',
 '8.36',
 '8.93',
 '8.42',
 '8.75',
 '',
 '6.17',
 '8.97',
 '7.06',
 '7.32',
 '',
 '8.29',
 '7.03',
 '8.41',
 '6.19',
 '5.19',
 '0.00',
 '0.00',
 '',
 '9.23',
 '4.23',
 '9.10',
 '8.94',
 '',
 '8.98',
 '7.70',
 '4.43',
 '9.05',
 '9.11',
 '9.30',
 '8.89',
 '9.43',
 '9.31',
 '8.14',
 '6.50',
 '8.10',
 '9.16',
 '8.42',
 '8.58',
 '8.89',
 '9.07',
 '7.72',
 '8.06',
 '6.27',
 '9.21',
 '8.97',
 '8.78',
 '6.91',
 '8.15',
 '6.48',
 '6.95',
 '8.54',
 '7.55',
 '6.28',
 '9.34',
 '9.00',
 '8.66',
 '',
 '8.30',
 '8.78',
 '7.99',
 '6.70',
 '6.75',
 '8.69',
 '9.06',
 '6.01',
 '9.26',
 '8.91',
 '8.85',
 '9.26',
 '5.19',
 '9.03',
 '8.97',
 '5.29',
 '7.86',
 '9.03',
 '8.06',
 '6.92',
 '9.41',
 '9.02',
 '6.91',
 '7.48',
 '8.89',
 '8.36',
 '8.74',
 '7.12',
 '6.89',
 '7.90',
 '6.50',
 '8.85',
 '8.46',
 '8.72',
 '9.02',
 '8.11',
 '8.11',
 '6.51',
 '7.71',
 '8.30',
 '',
 '8.77',
 '6.39',
 '8.93',
 '9.22',
 '7.94',
 '9.15',
 '3.61',
 '4.67',
 '7.78',
 '9.10',
 '7.68',
 '7.54'

In [95]:
#old_content > ul.search_list_1 > li > p > a > img
movies[movies['영화명']=='극장판 프리! - 타임리스 메들리 - 약속']

,영화명,개봉일,매출액,매출액점유율,관객수,스크린수,상영횟수,대표국적,등급,장르,제작연도,유형,감독,편당관객수,스크린당관객수


In [97]:
movies.to_excel('18~21영화통합.xlsx',index=False)